In [ ]:

import torch
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer

# 1. Load a pre-trained coreference resolution pipeline.
# This model will identify all mentions of entities and group them.
try:
    coref_pipeline = pipeline("coreference-resolution", model="coref-bert-large")
except Exception as e:
    print(f"Error loading coreference model: {e}")
    coref_pipeline = None

# 2. Load the fine-tuned T5 model for textual entailment.
# Assuming you have already fine-tuned and saved a model for this task.
try:
    entailment_model_path = "./fine_tuned_entailment_model"
    entailment_tokenizer = T5Tokenizer.from_pretrained(entailment_model_path)
    entailment_model = T5ForConditionalGeneration.from_pretrained(entailment_model_path)
except Exception as e:
    print(f"Error loading entailment model: {e}")
    entailment_model = None

def analyze_legal_document(legal_text):
    """
    Simulates the ClauseFinder analysis pipeline.
    """
    print("--- 📝 Performing Legal Document Analysis ---")
    
    # Step A: Coreference Resolution
    if coref_pipeline:
        print("\nStep 1: Coreference Resolution")
        coref_results = coref_pipeline(legal_text)
        for cluster in coref_results['clusters']:
            print(f"  - Entity Cluster: {cluster}")
    else:
        print("  - Coreference model not loaded. Skipping this step.")

    # Step B: Textual Entailment Analysis
    if entailment_model:
        print("\nStep 2: Textual Entailment Analysis")
        # Define a list of legal clauses and hypotheses to check.
        # In a real-world scenario, these would be loaded from a database.
        analysis_tasks = [
            {
                "clause": "This Agreement shall be governed by the laws of the State of Delaware.",
                "hypothesis": "The contract is subject to the jurisdiction of New York."
            },
            {
                "clause": "Party A shall provide a guarantee for the services rendered.",
                "hypothesis": "Party A is obligated to provide a guarantee."
            }
        ]

        for task in analysis_tasks:
            premise = task['clause']
            hypothesis = task['hypothesis']
            input_text = f"mnli premise: {premise} hypothesis: {hypothesis}"
            input_ids = entailment_tokenizer(input_text, return_tensors="pt").input_ids
            
            with torch.no_grad():
                outputs = entailment_model.generate(input_ids)
            
            entailment_result = entailment_tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            print(f"  - Clause: '{premise}'")
            print(f"    Hypothesis: '{hypothesis}'")
            print(f"    Result: '{entailment_result}'\n")
    else:
        print("  - Entailment model not loaded. Skipping this step.")




In [ ]:
# Example legal text for analysis
legal_document = """
This Agreement is made as of January 1, 2025, by and between ABC Corp. ("the Company") and Jane Doe ("the Consultant"). The Consultant agrees to provide services as defined herein. The Company's obligations include a monthly fee payment. Its guarantee to Jane Doe is non-negotiable. This Agreement is governed by the laws of the State of Delaware.
"""

# Run the analysis pipeline on the legal document.
analyze_legal_document(legal_document)
